In [2]:
import pandas as pd
import os
# import seaborn as sns
import numpy as np
import re
from datetime import datetime
path = './' + 'data/'
os.listdir(path)
# os.cpu_count()

['OD集货量',
 '中转场历史到件量0601至0831.csv',
 '场地数据.csv',
 '场地数据.xlsx',
 '班次数据0601至0831.csv',
 '运力配载数据0601至0831']

In [43]:
# %%time
# trans = pd.read_json('./data/运力配载数据0601至0831/line_info_0601.json', lines=True)
hist_arr = pd.read_csv(path + '中转场历史到件量0601至0831.csv')
# loc = pd.read_csv(path + '场地数据.csv')
# # loc.to_csv(path + '场地数据.csv', index = None, encoding='utf_8_sig')
# task_arrange = pd.read_csv(path + '班次数据0601至0831.csv')


path_od = path + 'OD集货量/'  
# path+
dfs = pd.DataFrame()
for fname in os.listdir(path_od):
    if re.search(r'\.csv$', fname):
        dfs = pd.concat(
            [dfs , pd.read_csv(path_od + fname)], axis = 0, ignore_index = True)
prc_in = pd.DataFrame(dfs) 
print(
    'loc:', loc.shape, '\nhist:' ,
    hist_arr.shape, '\ntask:',
    task_arrange.shape, '\nod:' ,
    dfs.shape)

NameError: name 'loc' is not defined

In [3]:
task_loc = pd.read_pickle('./data_clean/task_loc.p')

---

# Loc table clean  

In [3]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
loc[loc['场地代码'] == '552W']

,分拨区,所在城市名称,所在城市代码,场地名称,场地代码,场地类型,应用场景,主服务城市,主服务业务区,所在区部,单小时处理能力,卸货卡位总数,卸车卡位（17.5M）,卸车卡位（>=15M）,卸车卡位（<15M）,装货卡位总数,装车卡位（17.5M）,装车卡位（15M）,装车卡位（13.5M）,装车卡位（9.6M）,装车卡位（7.6M）,装车卡位（5.2M）
205,222Y,蚌埠市,552,蚌埠龙子湖中转场,552W,片区中转场,通用,552,551Y,222Y,21516,12,12,0,0,87,10,0,0,28,5,44


In [56]:
# loc.columns
loc = loc.drop(['所在城市代码','应用场景', '分拨区' , '所在城市名称', '场地名称', '主服务城市','所在区部'], axis = 1) 
loc[[ '卸车卡位（17.5M）', '卸车卡位（>=15M）', '卸车卡位（<15M）', '装车卡位（17.5M）', '装车卡位（15M）',
       '装车卡位（13.5M）', '装车卡位（9.6M）', '装车卡位（7.6M）', '装车卡位（5.2M）']]  = loc[[ '卸车卡位（17.5M）', '卸车卡位（>=15M）', '卸车卡位（<15M）', '装车卡位（17.5M）', '装车卡位（15M）',
       '装车卡位（13.5M）', '装车卡位（9.6M）', '装车卡位（7.6M）', '装车卡位（5.2M）']] == 0

In [16]:
# loc[['装车卡位（13.5M）', '装车卡位（9.6M）', '装车卡位（7.6M）', '装车卡位（5.2M）']].describe()
# loc.head(2)

In [76]:
loc['off_1'] = loc['卸车卡位（17.5M）'] * 17.5
loc['off_2'] = loc['卸车卡位（>=15M）'] * 15
loc['off_3'] = loc['卸车卡位（<15M）'] * (13.5 * 0.970721 + 9.6 * 11.056306 + 7.6 * 2.279279 + 5.2 * 9.826577)/ \
    (0.970721 + 11.056306 + 2.279279 + 9.826577)
loc['onn_1'] = loc['装车卡位（17.5M）'] * 17.5
loc['onn_2'] = loc['装车卡位（15M）'] * 15
loc['onn_3'] = loc['装车卡位（13.5M）'] * 13.5
loc['onn_4'] = loc['装车卡位（9.6M）'] * 9.6
loc['onn_5'] = loc['装车卡位（7.6M）'] * 7.6
loc['onn_6'] = loc['装车卡位（5.2M）'] * 5.2
loc['off'] = loc['off_1'] + loc['off_2'] + loc['off_3']
loc['onn'] = loc['onn_1'] + loc['onn_2'] + loc['onn_3'] + loc['onn_4'] + loc['onn_5'] + loc['onn_6']
loc = loc.drop(['off_1','off_2', 'off_3', 'onn_1', 'onn_2', 'onn_3', 'onn_4', 'onn_5', 'onn_6'], axis = 1)

loc['off_p1'] = loc['off']/loc['卸货卡位总数']
loc['off_p2'] = loc['off']/loc['装货卡位总数']
loc['onn_p1'] = loc['onn']/loc['卸货卡位总数']
loc['onn_p2'] = loc['onn']/loc['装货卡位总数']

In [128]:
# loc.to_pickle('./data_clean/loc.p')
# pd.read_pickle('./data_clean/loc.p')

In [82]:
corr_loc = loc.corr()
corr_loc.style.background_gradient(cmap='coolwarm', axis=None)


,单小时处理能力,卸货卡位总数,卸车卡位（17.5M）,卸车卡位（>=15M）,卸车卡位（<15M）,装货卡位总数,装车卡位（17.5M）,装车卡位（15M）,装车卡位（13.5M）,装车卡位（9.6M）,装车卡位（7.6M）,装车卡位（5.2M）,off,onn,off_p1,off_p2,onn_p1,onn_p2
单小时处理能力,1.000000,0.694292,-0.213250,-0.248743,0.140098,0.515413,-0.314467,-0.201609,-0.124173,-0.018013,-0.122168,-0.107542,-0.340977,-0.400161,-0.443044,-0.384526,-0.445735,-0.377653
卸货卡位总数,0.694292,1.000000,-0.281807,-0.236958,0.148402,0.561361,-0.405929,-0.138989,-0.105247,-0.002723,-0.060192,-0.131470,-0.384069,-0.394709,-0.587899,-0.475927,-0.599649,-0.472486
卸车卡位（17.5M）,-0.213250,-0.281807,1.000000,-0.180776,-0.234207,-0.228785,0.508240,-0.129468,0.180945,-0.230001,-0.078532,0.098776,0.567338,0.200912,0.226928,0.156419,0.170245,0.125439
卸车卡位（>=15M）,-0.248743,-0.236958,-0.180776,1.000000,-0.350540,-0.139530,0.025541,0.385759,0.116778,0.017047,0.073878,0.027772,0.596186,0.305287,0.402602,0.306679,0.313934,0.237445
卸车卡位（<15M）,0.140098,0.148402,-0.234207,-0.350540,1.000000,0.188829,-0.155749,-0.114385,0.067248,0.368717,0.185574,0.116768,-0.047610,0.057440,-0.215582,-0.147595,-0.168092,-0.112701
装货卡位总数,0.515413,0.561361,-0.228785,-0.139530,0.188829,1.000000,-0.327459,-0.111249,-0.184515,0.037481,-0.049992,-0.440404,-0.231441,-0.395207,-0.451163,-0.448590,-0.471082,-0.454295
装车卡位（17.5M）,-0.314467,-0.405929,0.508240,0.025541,-0.155749,-0.327459,1.000000,-0.227340,0.244645,-0.155300,-0.026065,0.052813,0.378318,0.487021,0.271116,0.207883,0.270907,0.195780
装车卡位（15M）,-0.201609,-0.138989,-0.129468,0.385759,-0.114385,-0.111249,-0.227340,1.000000,0.024345,-0.149801,0.117706,0.053725,0.189454,0.443154,0.212568,0.165900,0.243910,0.176107
装车卡位（13.5M）,-0.124173,-0.105247,0.180945,0.116778,0.067248,-0.184515,0.244645,0.024345,1.000000,0.078580,0.166986,0.233030,0.290666,0.578570,0.142412,0.148731,0.168714,0.155687
装车卡位（9.6M）,-0.018013,-0.002723,-0.230001,0.017047,0.368717,0.037481,-0.155300,-0.149801,0.078580,1.000000,0.253774,-0.046161,-0.004974,0.243038,0.079440,0.116319,0.129136,0.145468


# Task arrage clean

In [78]:
task_arrange[(task_arrange['operate_zone_code'] == '552W') & (task_arrange['batch_date'] == '2021-06-04')]

,operate_zone_code,batch_code,plan_begin_tm,plan_end_tm,batch_date
66107,552W,552W0301,03:01,06:30,2021-06-04
66108,552W,552W0730,07:30,09:00,2021-06-04
66109,552W,552W0930,09:30,11:00,2021-06-04
66110,552W,552W1101,11:01,13:10,2021-06-04
66111,552W,552W1520,15:20,16:40,2021-06-04
66112,552W,552W1830,18:30,21:50,2021-06-04
66113,552W,552W2200,22:00,03:00,2021-06-04


In [19]:
%%time

ta = task_arrange.groupby(['operate_zone_code', 'batch_date']).\
    agg(min_begin = ('plan_begin_tm', min),
    max_begin = ('plan_begin_tm', max),
    min_end = ('plan_end_tm', min),
    max_end = ('plan_end_tm', max),
    count = ('batch_code', 'nunique')
    
    ).reset_index()

Wall time: 41.7 s


In [103]:
def convt_min(col):
    ta[col] = ta[col].apply(lambda i: datetime.strptime(i,'%H:%M').hour*60 + datetime.strptime(i,'%H:%M').minute)
     
for i in ['min_begin', 'max_begin', 'min_end','max_end']:
    convt_min(i)
ta['time_consume_per_vote'] = (ta['max_end'] - ta['min_begin'])/ta['count']
 

In [4]:
ta.head()

,operate_zone_code,batch_date,min_begin,max_begin,min_end,max_end,count,time_consume_per_vote
0,010R,2021-06-01,1260,1400,370,1360,2,50.0
1,010R,2021-06-02,1260,1400,370,1360,2,50.0
2,010R,2021-06-03,1260,1400,370,1360,2,50.0
3,010R,2021-06-04,1260,1400,370,1360,2,50.0
4,010R,2021-06-05,1260,1400,370,1360,2,50.0


In [5]:
corr_ta = ta.corr()
corr_ta.style.background_gradient(cmap='coolwarm', axis=None)


,min_begin,max_begin,min_end,max_end,count,time_consume_per_vote
min_begin,1.000000,0.082276,0.370383,-0.547887,-0.614004,-0.711813
max_begin,0.082276,1.000000,-0.547942,0.151572,0.506862,-0.416152
min_end,0.370383,-0.547942,1.000000,0.129046,-0.577872,0.298075
max_end,-0.547887,0.151572,0.129046,1.000000,0.564497,0.712461
count,-0.614004,0.506862,-0.577872,0.564497,1.000000,0.187662
time_consume_per_vote,-0.711813,-0.416152,0.298075,0.712461,0.187662,1.000000


In [119]:
# pd.to_pickle(ta,  './data_clean/task.p')
# test = pd.read_pickle('./data_clean/task.p')

In [22]:
task_loc = ta.merge(loc, left_on='operate_zone_code', right_on = '场地代码', how = 'left')

In [31]:
# del loc, ta
# task_loc.to_pickle('./data_clean/task_loc.p')


In [37]:
task_loc_valued = task_loc[~task_loc['场地代码'].isna()]
task_loc_nulled = task_loc[task_loc['场地代码'].isna()]

In [38]:
# task_loc.isnull().sum()/len(task_loc)

---
# od

In [49]:
[prc_in[(prc_in['date1'] == '2021-06-08') & (prc_in['destsitecode'] == '552W')].sum(),
prc_in[(prc_in['date1'] == '2021-06-04') & (prc_in['srcsitecode'] == '552W')].sum()]

[srcbatchcode    010R2100010R2320010R2320010R2320010W0440010W13...
 srcsitecode     010R010R010R010R010W010W010W010W010W010W010W01...
 destsitecode    552W552W552W552W552W552W552W552W552W552W552W55...
 route_code      T4T4T6ZT4T6T6T6T6T4T6T77T6T4T6T77T4T6T4T6T4T6T...
 date1           2021-06-082021-06-082021-06-082021-06-082021-0...
 votes                                                     42199.0
 weight1                                             139301.773369
 air_votes                                                  1908.0
 air_weight1                                               4656.48
 dtype: object,
 srcbatchcode    552W0301552W0301552W0301552W0301552W0301552W03...
 srcsitecode     552W552W552W552W552W552W552W552W552W552W552W55...
 destsitecode    010WB010WE010WS020W020W020WE020WM021WG021WG021...
 route_code      T77T6ZT6T6T77T77T77T77T8T77T6T77ZT6T6T77T6T77T...
 date1           2021-06-042021-06-042021-06-042021-06-042021-0...
 votes                                        

In [52]:
# prc_in.head()

od = prc_in.groupby(['srcsitecode','destsitecode', 'date1']).agg(
    batch_cnt = ('srcbatchcode', 'nunique'),
    route_cnt = ('route_code', 'nunique'),
    votes = ('votes', sum),
    weights = ('weight1', sum),
    air_votes =('air_votes', sum),
    air_weights = ('air_weight1', sum)
).reset_index()

In [103]:
# od.to_pickle('./data_clean/od.p')

In [96]:
od.columns

Index(['srcsitecode', 'destsitecode', 'date1', 'batch_cnt', 'route_cnt',
       'votes', 'weights', 'air_votes', 'air_weights'],
      dtype='object')

In [100]:
od.head()

,srcsitecode,destsitecode,date1,batch_cnt,route_cnt,votes,weights,air_votes,air_weights
0,010R,010R,2021-06-01,2,4,10.0,122.500000,2.0,4.166667
1,010R,010R,2021-06-02,2,3,4.0,32.500000,0.0,0.000000
2,010R,010R,2021-06-03,2,4,15.0,102.416667,2.0,2.000000
3,010R,010R,2021-06-04,2,5,27.0,193.175000,2.0,2.900000
4,010R,010R,2021-06-05,1,2,2.0,2.000000,0.0,0.000000


---

# hist

In [44]:
hist_arr[(hist_arr['a.zonecode'] == '552W') &  ( hist_arr['a.report_dt'] == '2021-06-04' )]

,a.report_dt,a.zonecode,all_arr_waybill,all_arr_weight,simu_arr_waybill,simu_arr_weight
35110,2021-06-04,552W,175334,472758.782153,6500,13488.254667


In [102]:
# hist_arr.to_pickle('./data_clean/hist_arr.p')

In [88]:
train = task_loc.merge(hist_arr, \
    left_on = ['operate_zone_code', 'batch_date'], right_on=['a.zonecode', 'a.report_dt'],
    how = 'right')

In [89]:
train

,operate_zone_code,batch_date,min_begin,max_begin,min_end,max_end,count,time_consume_per_vote,场地代码,场地类型,...,off_p1,off_p2,onn_p1,onn_p2,a.report_dt,a.zonecode,all_arr_waybill,all_arr_weight,simu_arr_waybill,simu_arr_weight
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2021-06-08,010CC006,525,1280.400000,0,0.000000
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2021-06-08,010CC007,0,0.000000,0,0.000000
2,010R,2021-06-08,1260.0,1400.0,370.0,1360.0,2.0,50.0,NaN,NaN,...,NaN,NaN,NaN,NaN,2021-06-08,010R,102944,230415.243664,76818,165515.803556
3,010RA,2021-06-08,30.0,791.0,160.0,910.0,4.0,220.0,NaN,NaN,...,NaN,NaN,NaN,NaN,2021-06-08,010RA,109154,249089.557278,109154,249089.557278
4,010RG,2021-06-08,30.0,1320.0,20.0,1020.0,4.0,247.5,NaN,NaN,...,NaN,NaN,NaN,NaN,2021-06-08,010RG,119183,261849.663955,118678,260439.168955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2021-08-15,P891CDA,0,0.000000,0,0.000000
61527,S028WJ,2021-08-15,1320.0,1320.0,420.0,420.0,1.0,-900.0,NaN,NaN,...,NaN,NaN,NaN,NaN,2021-08-15,S028WJ,426,7779.334356,0,0.000000
61528,S551WJ,2021-08-15,1320.0,1320.0,420.0,420.0,1.0,-900.0,NaN,NaN,...,NaN,NaN,NaN,NaN,2021-08-15,S551WJ,45,1048.212836,0,0.000000
61529,SGN01R,2021-08-15,420.0,1110.0,510.0,1320.0,4.0,225.0,NaN,NaN,...,NaN,NaN,NaN,NaN,2021-08-15,SGN01R,0,0.000000,0,0.000000


In [91]:
# od

---
# transport


In [67]:
trans.head(2)

,cvyName,cvyType,lineCode,lineKey,lineRequireDate,passZoneCodes,stopOver,taskInfos,transportLevel,vehicleInfos
0,穗北官窑0215（常规）,4,020WE757WA0215,"020WE757WA0215_2021-06-07_020WE,757WA",2021-06-07,"[{'arriveBatchCode': '757WA0301', 'arriveTm': ...",False,[{'drivingFactors': 'T6|757WA|757WL|758|766;T7...,2,"[{'extraMinLoadWeight': 3900000.1549720764, 'l..."
1,中重马坡0800,4,760WH010WS0800,"760WH010WS0800_2021-06-11_760WH,010WS",2021-06-11,"[{'arriveBatchCode': '010WS1801', 'arriveTm': ...",False,[{'drivingFactors': 'T6;T77;T4;T1;T8;T29|010R|...,1,"[{'extraMinLoadWeight': 7200000.286102295, 'lo..."


In [86]:
trans.iloc[11,:]['passZoneCodes'][0]

{'arriveBatchCode': '632VA1200',
 'arriveTm': 1622693460000,
 'destZoneCode': '632VA',
 'order': 1,
 'sendBatchCode': '536XA2359',
 'sendTm': 1622671200000,
 'sendZoneCode': '536XA'}

In [87]:
trans.iloc[11,:]['taskInfos'][0] # ['unloadZone']

{'drivingFactors': 'T6|632VA;T77|632VA;T4|632VA;T1;T8;T29|632VA;ZT4|632VA;ZT6|632VA',
 'loadZone': '536XA',
 'taskId': '536XA_632VA',
 'unloadZone': '632VA'}

In [81]:
trans.iloc[1,:]['vehicleInfos'][0] # ['unloadZone']

{'extraMinLoadWeight': 7200000.286102295,
 'loadWeight': 12000000.0,
 'vehicleType': {'l': 16.0,
  't': 30.0,
  'tonnage': '30T',
  'vehicleLength': '16M'}}